# Compute i coefficients

In [320]:
import brightway2 as bw
import pandas as pd
import os

# Set working directry
path = "."
os.chdir(path)

# Local functions
from utils.lookup_func import lookup_geothermal

# Set project
bw.projects.set_current("Geothermal")

# Retrieve activities
wellhead, diesel, steel, cement, water, \
drilling_mud, drill_wst, wells_closr, coll_pipe, \
plant, ORC_fluid, ORC_fluid_wst, diesel_stim, co2,_, _ = lookup_geothermal()
cooling_tower=bw.Database("geothermal energy").search("cooling tower")[0].key

list_act = [wellhead, diesel, steel, cement, water, 
         drilling_mud, drill_wst, wells_closr, coll_pipe,
         plant, cooling_tower, ORC_fluid, ORC_fluid_wst, diesel_stim]

# Retrieve methods 
ILCD_CC = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "climate change total" in str(method)]
ILCD_HH = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "human health" in str(method)]
ILCD_EQ = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "ecosystem quality" in str(method)]
ILCD_RE = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "resources" in str(method)]
ILCD = ILCD_CC + ILCD_HH + ILCD_EQ + ILCD_RE

# Calculate impact of activities
lca = bw.LCA({list_act[0]: 1}, ILCD[0])
lca.lci()
lca.lcia()
coeff = {}
for method in ILCD:
    s=[]
    lca.switch_method(method)
    for act in list_act:
        lca.redo_lcia({act: 1})
        s.append(lca.score)
    coeff[method] = s

# Retrieve CF for co2 emissions    
for method in ILCD:
    CFs = bw.Method(method).load()
    coeff[method].append(next((cf[1] for cf in CFs if cf[0]==co2),0))

# Build matrix
col_names = ["wellhead", "diesel", "steel", "cement", "water", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid", "ORC_fluid_wst", "diesel_stim", "co2"]
coeff_matrix = pd.DataFrame.from_dict(coeff, orient="index", columns=col_names)

# Re-arrange matrix
coeff_matrix["concrete"] = coeff_matrix["cement"] + coeff_matrix["water"] * 1/0.65
coeff_matrix["ORC_fluid_tot"] = coeff_matrix["ORC_fluid"] - coeff_matrix["ORC_fluid_wst"]
coeff_matrix["electricity_stim"] = coeff_matrix["diesel_stim"] * 3.6
coeff_matrix["drill_wst"] = coeff_matrix["drill_wst"] * -1
coeff_matrix=coeff_matrix.drop(columns=["cement", "ORC_fluid", "ORC_fluid_wst", "diesel_stim"])

col_ord = ["wellhead", "diesel", "steel", "concrete", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid_tot", "water", "electricity_stim", "co2"]
coeff_matrix=coeff_matrix[col_ord]

is_= ["i1", "i2.1", "i2.2", "i2.3", "i2.4", "i2.5", \
      "i2.6","i3", "i4.1", "i4.2", "i4.3", "i5.1", "i5.2", "i6"]

if len(coeff_matrix.columns) == len(is_):
    coeff_matrix.columns=is_

# Define symbolic

In [321]:
from sympy import symbols, collect, ratsimp, fraction

In [322]:
# Coefficients
i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6 = \
symbols('i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6')

# Other variables that are actually irrelevant, because they will be multiplied by 0 in the enhanced model
# I wanna keep them though for the sake of completeness
SW_n, S_w, S_el = symbols('SW_n, SW, S_el')

# Main parameters
P_ne, CW_ne, PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP = \
symbols('P_ne, CW_ne, PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP')

# Constants
CT_el, CT_n, W_en, OF,DW = symbols ('CT_el, CT_n, W_E_en, OF, DW')


# Main equation

In [323]:
# Supporting equations for CONVENTIONAL geothermal
W_Pn = P_ne/CW_ne             # production wells
W_In = P_ne/CW_ne / PIratio  # injection wells
W_Mn = W_Pn*D_i*LT            # makeup wells

# Number of wells with success rate
W_n_sr = W_Pn/(SR_p/100) + W_In/(SR_p/100) + W_Mn/(SR_m/100)
W_n = W_Pn + W_In + W_Mn
W_E_en = W_en
W_E_en_sr = W_en/(SR_e/100)

In [324]:
# Equation
nominator = (W_n + W_E_en) *i1 \
          + W_d* (W_n_sr + W_E_en_sr) * (D*i2_1 + C_S*i2_2 + C_C*i2_3  +  DM*i2_4 + DW*i2_5 + i2_6) + \
          + W_n*CP*i3 \
          + P_ne*i4 \
          + SW_n*S_w * (i5_1 + S_el*i5_2)

c1 = P_ne*CF*(1-AP)*LT*8760000
d1 = CT_el*CT_n*1000*LT
denominator = c1  -  d1

summand = E_co2*i6

eq = nominator/denominator + summand
eq = eq.subs(i4, i4_1 + i4_2*CT_n + i4_3*OF)

In [325]:
# Show equation
eq

E_co2*i6 + (CP*i3*(D_i*LT*P_ne/CW_ne + P_ne/CW_ne + P_ne/(CW_ne*PIratio)) + P_ne*(CT_n*i4_2 + OF*i4_3 + i4_1) + SW*SW_n*(S_el*i5_2 + i5_1) + W_d*(100*W_E_en/SR_e + 100*D_i*LT*P_ne/(CW_ne*SR_m) + 100*P_ne/(CW_ne*SR_p) + 100*P_ne/(CW_ne*PIratio*SR_p))*(C_C*i2_3 + C_S*i2_2 + D*i2_1 + DM*i2_4 + DW*i2_5 + i2_6) + i1*(W_E_en + D_i*LT*P_ne/CW_ne + P_ne/CW_ne + P_ne/(CW_ne*PIratio)))/(8760000*CF*LT*P_ne*(1 - AP) - 1000*CT_el*CT_n*LT)

# Get parameters values

In [326]:
from cge_klausen import get_parameters
parameters=get_parameters()
parameters.static()    

# Rename parameters (in accordance with equation)

# Parameters of simplified models
CW_ne_val    = parameters["gross_power_per_well"]
E_co2_val    = parameters["co2_emissions"]
D_i_val      = parameters["initial_harmonic_decline_rate"]
#SR_p_val     = parameters["success_rate_primary_wells"]
SR_p_val     = 100
W_d_val      = parameters["average_depth_of_wells"]

# Parameters to be fixed
P_ne_val     = parameters["installed_capacity"]
PIratio_val  = parameters["production_to_injection_ratio"]
LT_val       = parameters["lifetime"]
#SR_m_val     = parameters["success_rate_makeup_wells"]
SR_m_val     = 100
#SR_e_val     = parameters["success_rate_exploration_wells"]
SR_e_val     = 100
D_val        = parameters["specific_diesel_consumption"]
C_S_val      = parameters["specific_steel_consumption"]
C_C_val      = parameters["specific_cement_consumption"]
DM_val       = parameters["specific_drilling_mud_consumption"]
CP_val       = parameters["collection_pipelines"]
CF_val       = parameters["capacity_factor"]
AP_val       = parameters["auxiliary_power"]

# Constants
CT_el_val  = 864
CT_n_val   = 7/303.3
W_en_val   = 3 * 0.3
OF_val     = 0
DW_val     = 450

par_dict = { P_ne: P_ne_val,
             PIratio: PIratio_val,
             LT: LT_val,
             SR_m: SR_m_val,
             SR_e: SR_e_val,
             D: D_val,
             C_S: C_S_val,
             C_C: C_C_val,
             DM: DM_val,
             CP: CP_val,
             CF: CF_val,
             AP: AP_val,
             CT_el: CT_el_val,
             CT_n: CT_n_val,
             W_en: W_en_val,
             OF: OF_val,
             DW: DW_val}

# Groups of environmental categories

In [327]:
group1 = ["climate change total"]

group2 = ["carcinogenic effects", "ionising radiation", "non-carcinogenic effects", 
          "ozone layer depletion", "photochemical ozone creation", "respiratory effects, inorganics",
          "freshwater and terrestrial acidification", "freshwater ecotoxicity", 
          "freshwater eutrophication", "marine eutrophication", "terrestrial eutrophication",
          "minerals and metals", "dissipated water", "fossils", "land use"]

# Simplified equations

In [328]:
# Define symbols
alpha_1, alpha_2, beta_1, beta_2, beta_3, beta_4, beta_5, beta_6 = \
symbols('alpha_1, alpha_2, beta_1, beta_2, beta_3, beta_4, beta_5, beta_6')

## Group 1 (Climate Change)

In [329]:
eq_group1_all_thresholds = alpha_1*E_co2 + alpha_2
eq_group1_all_thresholds

E_co2*alpha_1 + alpha_2

## Group 2 

### Threshold 15/20%

In [330]:
eq_group2_15_20 = (W_d*beta_1 + beta_2)/CW_ne + CW_ne*beta_3 + beta_4
eq_group2_15_20

CW_ne*beta_3 + beta_4 + (W_d*beta_1 + beta_2)/CW_ne

## Threshold 10%

In [331]:
eq_group2_10 = (D_i*W_d*beta_1 + D_i*beta_2 + W_d*beta_3 + beta_4) / (CW_ne) + W_d*beta_5 + beta_6
eq_group2_10 

W_d*beta_5 + beta_6 + (D_i*W_d*beta_1 + D_i*beta_2 + W_d*beta_3 + beta_4)/CW_ne

## Threshold 5%

In [332]:
eq_group2_05 = (D_i*SR_p*W_d*beta_1 + D_i*SR_p*beta_2 + SR_p*beta_3 + W_d*beta_4) / (CW_ne*SR_p) + W_d*beta_5 + beta_6
eq_group2_05 

W_d*beta_5 + beta_6 + (D_i*SR_p*W_d*beta_1 + D_i*SR_p*beta_2 + SR_p*beta_3 + W_d*beta_4)/(CW_ne*SR_p)

# Compute alphas 

## All thresholds

In [333]:
# Replace parameters in equation
eq_alpha = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        D_i: D_i_val,
        CW_ne: CW_ne_val,
        W_d: W_d_val}
eq_alpha = eq_alpha.subs(repl)

alpha_dict = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group1:
        eq_alpha_is = eq_alpha.subs(is_dict)
        alpha1 = collect(eq_alpha_is, E_co2, evaluate=False)[E_co2]
        alpha2 = collect(eq_alpha_is, E_co2, evaluate=False)[1]
        alpha_dict[method] = {'alpha1': alpha1, 'alpha2': alpha2}
           

In [334]:
# Example of the formula
eq_alpha_is

1.0*E_co2 + 0.0037629428941863

In [335]:
alpha_df = pd.DataFrame.from_dict(alpha_dict, orient='index')
alpha_df = alpha_df.astype(float)
alpha_df_5 = alpha_df_10 = alpha_df_15 = alpha_df_20 = alpha_df
alpha_df

,,,alpha1,alpha2
ILCD 2.0 2018 midpoint no LT,climate change,climate change total,1.0,0.003763


# Compute betas

## Threshold 15-20%

In [336]:
# Threshold = 15%/20%
eq_beta = eq.subs(par_dict)
eq_beta = eq_beta.subs(E_co2, E_co2_val)
repl = {SR_p: SR_p_val, D_i: D_i_val} 
eq_beta = eq_beta.subs(repl)
        
beta_dict_15 = {}     
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group2:
        eq_beta_is = eq_beta.subs(is_dict)
        eq_beta_is = ratsimp(eq_beta_is)
        temp = collect(eq_beta_is, [W_d,1/CW_ne], evaluate=False)
        beta1 = collect(temp[list(temp.keys())[1]], W_d, evaluate=False)[W_d]
        beta2 = collect(temp[list(temp.keys())[1]], W_d, evaluate=False)[1]
        beta3 = temp[W_d]
        beta4 = temp[1]
        
        beta_dict_15[method] = {'beta1': beta1, 'beta2': beta2,
                                     'beta3': beta3, 'beta4': beta4}


beta_dict_20 = beta_dict_15


In [337]:
# Show example of the formula
eq_beta_is

4.03652539305323e-13*W_d + 3.4049199784091e-8 + 1.0*(9.41855925045754e-11*W_d + 1.50354039452095e-7)/CW_ne

In [338]:
beta_df_15 = pd.DataFrame.from_dict(beta_dict_15, orient='index')
beta_df_15 = beta_df_15.astype(float)

#20% and 15% threshold have the same configuration
beta_df_20 = beta_df_15

beta_df_15

beta1  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      1.362981e-12   
                                               ionising radiation                        2.375533e-07   
                                               non-carcinogenic effects                  3.685078e-12   
                                               ozone layer depletion                     7.749590e-13   
                                               photochemical ozone creation              6.153569e-08   
                                               respiratory effects, inorganics           3.104300e-13   
                             ecosystem quality freshwater and terrestrial acidification  5.247697e-08   
                                               freshwater ecotoxicity                    4.149394e-05   
                                               freshwater eutrophication                 2.680958e-10   
                                               marine eutrophication                     1.977310e-08   
                                               terrestrial eutrophication                2.146585e-07   
                             resources         dissipated water                          1.617301e-06   
                                               fossils                                   8.367748e-05   
                                               land use                                  2.259963e-05   
                                               minerals and metals                       9.418559e-11   

                                                                                                beta2  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      9.489278e-10   
                                               ionising radiation                        1.188166e-04   
                                               non-carcinogenic effects                  2.273578e-09   
                                               ozone layer depletion                     2.936341e-10   
                                               photochemical ozone creation              2.124991e-05   
                                               respiratory effects, inorganics           3.845425e-10   
                             ecosystem quality freshwater and terrestrial acidification  2.583111e-05   
                                               freshwater ecotoxicity                    1.205271e-02   
                                               freshwater eutrophication                 2.156094e-07   
                                               marine eutrophication                     5.226420e-06   
                                               terrestrial eutrophication                5.901544e-05   
                             resources         dissipated water                          1.787742e-03   
                                               fossils                                   6.015186e-02   
                                               land use                                  3.105897e-02   
                                               minerals and metals                       1.503540e-07   

                                                                                                beta3  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      5.841345e-15   
                                               ionising radiation                        1.018086e-09   
                                               non-carcinogenic effects                  1.579319e-14   
                                               ozone layer depletion                     3.321253e-15   
                                               photochemical ozone creation              2.637244e-10   
                                               respiratory effects, inorganics           1.330414e-15   

## Threshold = 10%

In [339]:
eq_beta_10 = eq.subs(par_dict)
eq_beta_10 = eq_beta_10.subs(E_co2, E_co2_val)
repl = {SR_p: SR_p_val} 
eq_beta_10 = eq_beta_10.subs(repl)
        
beta_dict_10 = {}   
    
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group2:
        eq_beta_10_is = eq_beta_10.subs(is_dict)
        eq_beta_10_is = ratsimp(eq_beta_10_is)
               
        temp   = collect(eq_beta_10_is, [W_d, 1/CW_ne,], evaluate=False)
        temp_2 = collect(temp[list(temp.keys())[1]], [D_i*W_d, D_i, W_d], evaluate=False)
        
        beta1  = temp_2[D_i*W_d]      
        beta2  = temp_2[D_i]
        beta3  = temp_2[W_d]
        beta4  = temp_2[1]
              
        beta5 = temp[W_d]
        beta6 = temp[1]
        
        beta_dict_10[method] = {'beta1': beta1, 'beta2': beta2,
                                     'beta3': beta3, 'beta4': beta4,
                                     'beta5': beta5, 'beta6': beta6}      
    

In [340]:
# Example of the formula
eq_beta_10_is

4.03652539305323e-13*W_d + 3.4049199784091e-8 + 1.0*(9.41855925045754e-10*D_i*W_d + 1.50354039452095e-6*D_i + 4.70927962522877e-11*W_d + 7.51770197260474e-8)/CW_ne

In [341]:
beta_df_10 = pd.DataFrame.from_dict(beta_dict_10, orient='index')
beta_df_10 = beta_df_10.astype(float)
beta_df_10

beta1  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      1.362981e-11   
                                               ionising radiation                        2.375533e-06   
                                               non-carcinogenic effects                  3.685078e-11   
                                               ozone layer depletion                     7.749590e-12   
                                               photochemical ozone creation              6.153569e-07   
                                               respiratory effects, inorganics           3.104300e-12   
                             ecosystem quality freshwater and terrestrial acidification  5.247697e-07   
                                               freshwater ecotoxicity                    4.149394e-04   
                                               freshwater eutrophication                 2.680958e-09   
                                               marine eutrophication                     1.977310e-07   
                                               terrestrial eutrophication                2.146585e-06   
                             resources         dissipated water                          1.617301e-05   
                                               fossils                                   8.367748e-04   
                                               land use                                  2.259963e-04   
                                               minerals and metals                       9.418559e-10   

                                                                                                beta2  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      9.489278e-09   
                                               ionising radiation                        1.188166e-03   
                                               non-carcinogenic effects                  2.273578e-08   
                                               ozone layer depletion                     2.936341e-09   
                                               photochemical ozone creation              2.124991e-04   
                                               respiratory effects, inorganics           3.845425e-09   
                             ecosystem quality freshwater and terrestrial acidification  2.583111e-04   
                                               freshwater ecotoxicity                    1.205271e-01   
                                               freshwater eutrophication                 2.156094e-06   
                                               marine eutrophication                     5.226420e-05   
                                               terrestrial eutrophication                5.901544e-04   
                             resources         dissipated water                          1.787742e-02   
                                               fossils                                   6.015186e-01   
                                               land use                                  3.105897e-01   
                                               minerals and metals                       1.503540e-06   

                                                                                                beta3  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      6.814903e-13   
                                               ionising radiation                        1.187767e-07   
                                               non-carcinogenic effects                  1.842539e-12   
                                               ozone layer depletion                     3.874795e-13   
                                               photochemical ozone creation              3.076785e-08   
                                               respiratory effects, inorganics           1.552150e-13   

## Threshold = 5%

In [342]:
eq_beta_5 = eq.subs(par_dict)
eq_beta_5 = eq_beta_5.subs(E_co2, E_co2_val)
        
beta_dict_5 = {}       
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group2:
        eq_beta_5_is = eq_beta_5.subs(is_dict)
        eq_beta_5_is = ratsimp(eq_beta_5_is)
               
        temp   = collect(eq_beta_5_is, [W_d, 1/CW_ne*SR_p], evaluate=False)
        temp_2 = collect(temp[list(temp.keys())[1]], [SR_p*W_d*D_i, D_i*SR_p, SR_p, W_d], evaluate=False)
        
        beta1  = temp_2[D_i*SR_p*W_d]
        beta2  = temp_2[D_i*SR_p]
        beta3  = temp_2[W_d]
        beta4  = temp_2[SR_p]
               
        beta5 = temp[W_d]
        beta6 = temp[1]
        
        beta_dict_5[method] = {'beta1': beta1, 'beta2': beta2,
                                     'beta3': beta3, 'beta4': beta4,
                                     'beta5': beta5, 'beta6': beta6}      
    

In [343]:
# Show example of the formula
eq_beta_5_is

4.03652539305323e-13*W_d + 3.4049199784091e-8 + 1.0*(9.41855925045754e-10*D_i*SR_p*W_d + 1.50354039452095e-6*D_i*SR_p + 7.51770197260474e-8*SR_p + 4.70927962522877e-9*W_d)/(CW_ne*SR_p)

In [344]:
beta_df_5 = pd.DataFrame.from_dict(beta_dict_5, orient='index')
beta_df_5 = beta_df_5.astype(float)
beta_df_5

beta1  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      1.362981e-11   
                                               ionising radiation                        2.375533e-06   
                                               non-carcinogenic effects                  3.685078e-11   
                                               ozone layer depletion                     7.749590e-12   
                                               photochemical ozone creation              6.153569e-07   
                                               respiratory effects, inorganics           3.104300e-12   
                             ecosystem quality freshwater and terrestrial acidification  5.247697e-07   
                                               freshwater ecotoxicity                    4.149394e-04   
                                               freshwater eutrophication                 2.680958e-09   
                                               marine eutrophication                     1.977310e-07   
                                               terrestrial eutrophication                2.146585e-06   
                             resources         dissipated water                          1.617301e-05   
                                               fossils                                   8.367748e-04   
                                               land use                                  2.259963e-04   
                                               minerals and metals                       9.418559e-10   

                                                                                                beta2  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      9.489278e-09   
                                               ionising radiation                        1.188166e-03   
                                               non-carcinogenic effects                  2.273578e-08   
                                               ozone layer depletion                     2.936341e-09   
                                               photochemical ozone creation              2.124991e-04   
                                               respiratory effects, inorganics           3.845425e-09   
                             ecosystem quality freshwater and terrestrial acidification  2.583111e-04   
                                               freshwater ecotoxicity                    1.205271e-01   
                                               freshwater eutrophication                 2.156094e-06   
                                               marine eutrophication                     5.226420e-05   
                                               terrestrial eutrophication                5.901544e-04   
                             resources         dissipated water                          1.787742e-02   
                                               fossils                                   6.015186e-01   
                                               land use                                  3.105897e-01   
                                               minerals and metals                       1.503540e-06   

                                                                                                beta3  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      6.814903e-11   
                                               ionising radiation                        1.187767e-05   
                                               non-carcinogenic effects                  1.842539e-10   
                                               ozone layer depletion                     3.874795e-11   
                                               photochemical ozone creation              3.076785e-06   
                                               respiratory effects, inorganics           1.552150e-11   

# Save to excel

In [314]:
with pd.ExcelWriter(os.path.join(path, 'generated_files\Simplified models coefficients conventional - symbolic - thresholds.xlsx')) as writer:
    alpha_df_5.to_excel(writer, sheet_name='alpha_5%')
    alpha_df_10.to_excel(writer, sheet_name='alpha_10%')
    alpha_df_15.to_excel(writer, sheet_name='alpha_15%')
    alpha_df_20.to_excel(writer, sheet_name='alpha_20%')
    beta_df_5.to_excel(writer, sheet_name='beta_5%')
    beta_df_10.to_excel(writer, sheet_name='beta_10%')
    beta_df_15.to_excel(writer, sheet_name='beta_15%')
    beta_df_20.to_excel(writer, sheet_name='beta_20%')

## Other save

In [345]:
acronyms_dict = {
    'climate change total' : 'CC',
    'carcinogenic effects' : 'HT-c',
    'ionising radiation' : 'IR' ,
    'non-carcinogenic effects' : 'HT-nc',
    'ozone layer depletion' : 'OD',
    'photochemical ozone creation': 'POC',
    'respiratory effects, inorganics': 'PM/RI',
    'freshwater and terrestrial acidification':'A',
    'freshwater ecotoxicity':'ET',
    'freshwater eutrophication':'Ef',
    'marine eutrophication':'Em',
    'terrestrial eutrophication':'Et',
    'dissipated water':'WU',
    'fossils':'RUe',
    'land use':'LU',
    'minerals and metals': 'RUm'}

In [346]:
beta_dict_15_ac=dict((acronyms_dict[k[2]], v) for (k, v) in beta_dict_15.items())
beta_df_15_ac = pd.DataFrame.from_dict(beta_dict_15_ac, orient="index")
beta_df_15_ac

,beta1,beta2,beta3,beta4
HT-c,1.36298051756007e-12,9.48927796539193e-10,5.84134507525743e-15,7.54883639674847e-11
IR,2.37553325009444e-7,0.000118816560384475,1.01808567861190e-9,2.05783785638447e-5
HT-nc,3.68507849697812e-12,2.27357827752467e-9,1.57931935584777e-14,2.83452005006882e-10
OD,7.74958967233194e-13,2.93634095763777e-10,3.32125271671368e-15,7.11468751378027e-11
POC,6.15356935344417e-8,2.12499131058050e-5,2.63724400861893e-10,2.70774426724808e-6
PM/RI,3.10429995727663e-13,3.84542520646098e-10,1.33041426740427e-15,4.51146604143829e-11
A,5.24769684039146e-8,2.58311125181477e-5,2.24901293159634e-10,4.40735300390936e-6
ET,4.14939363976364e-5,0.0120527113729358,1.77831155989870e-7,0.00132029981944263
Ef,2.68095755267315e-10,2.15609448404816e-7,1.14898180828849e-12,3.73829331925361e-8
Em,1.97731040373959e-8,5.22642026728891e-6,8.47418744459824e-11,8.32045972793645e-7


In [347]:
alpha_dict_ac=dict((acronyms_dict[k[2]], v) for (k, v) in alpha_dict.items())
alpha_df_ac = pd.DataFrame.from_dict(alpha_dict_ac, orient="index")
alpha_df_ac

,alpha1,alpha2
CC,1.00000000000000,0.00376294289418630


In [348]:
if SR_p_val == 100:
    file_name = 'Simplified conventional coefficients acronyms - NO SR.xlsx'
else:
    file_name = 'Simplified conventional coefficients acronyms.xlsx'

with pd.ExcelWriter(os.path.join(path, file_name)) as writer:
    beta_df_15_ac.to_excel(writer, sheet_name='beta_15%')
    alpha_df_ac.to_excel(writer, sheet_name='alpha')